# Charts for my website

- Author: Kiril from Mindgraph
- Last meaningful update: 30-11-2025

This notebook creates some `altair` charts for [my website](www.mindgraph.dk). The charts are displayed as visuals in this notebook and also exported to `.html` files on GitHub so that they can be directly embedded on the website.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import ColumnDataSource, Legend, HoverTool, NumeralTickFormatter
from bokeh.layouts import row
import hvplot.pandas  # activates hvPlot on DataFrames
import holoviews as hv
import panel as pn
import altair as alt

hv.extension("bokeh")

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
# Importing data [WIP - to be expanded]
ft_afsteminger = pd.read_parquet("output/ft_afstemninger.parquet")

In [3]:
# # Changing to the default color palette in plotly
# pio.templates.default = "plotly"

In [4]:
# Creating a custom color palette with the MG colors for plotly
# Using the "Classy" palette from: https://mycolor.space/?hex=%231EA2B5&sub=1
my_custom_palette = ["#1ea2b5", "#324b4f", "#95b0b5", "#9f8ac3", "#6b588d"]
my_template = pio.templates["plotly_white"].layout.template
my_template.layout.colorway = my_custom_palette
pio.templates["my_custom_template"] = my_template
pio.templates.default = "my_custom_template"

## Figure 1

INSERT CHART WITH % OF VOTES BY FOR/AGAINST/ETC ACROSS TIME - STACKED TO 100%

In [5]:
# Aggregating data for chart
id_cols = ["År", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afsteminger[ft_afsteminger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("År")["Sæson"].transform("count")
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)

# Pivoting the data
chart_data = chart_data.pivot_table(
    index="År", columns="Stemme", values="AndelStemmer", aggfunc="sum"
).reset_index()
chart_data = chart_data.fillna(0)

# Reordering the columns
cols_order = ["År", "For", "Imod", "Hverken for eller imod"]
chart_data = chart_data[cols_order]

In [6]:
# Creating a chart in Danish
cat_cols = ["For", "Imod", "Hverken for eller imod"]
fig = chart_data.hvplot.area(
    x="År",
    y=cat_cols,
    ylabel="Andel af alle stemmer",
    stacked=True,
    hover_cols="all",
    color=my_custom_palette,
    line_color=None,
    line_width=0,
)

# Wrapping the chart in a responsive HTML element and exporting
responsive_chart = pn.panel(fig, min_height=450, sizing_mode="scale_both")
responsive_chart.save(".charts/figure_01_dk.html", embed=True)
fig

:NdOverlay   [Stemme]
   :Area   [År]   (value,Baseline)

In [7]:
# Aggregating data for chart
id_cols = ["År", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afsteminger[ft_afsteminger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("År")["Sæson"].transform("count")
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)

In [8]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% af alle stemmer"}
chart_data_dk = chart_data.rename(columns=col_names)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Imod", "Hverken for eller imod"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_dk["CustomOrder"] = chart_data_dk["Stemme"].map(vote_order_mapping)

# Creating a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_area()
    .encode(
        x="År:O",
        y=alt.Y("% af alle stemmer:Q", axis=alt.Axis(format=".0%")),
        color=alt.Color(
            "Stemme:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=["År:O", "Stemme:N", alt.Tooltip("% af alle stemmer:Q", format=".0%")],
    )
)
chart.save(".charts/figure_01_dk.html")
chart

alt.Chart(...)

In [9]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% of all votes", "År": "Year", "Stemme": "Vote"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing certain values
val_mapping = {
    "For": "For",
    "Imod": "Against",
    "Hverken for eller imod": "Neither for nor against",
}
chart_data_en["Vote"] = chart_data_en["Vote"].replace(val_mapping)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Against", "Neither for nor against"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_en["CustomOrder"] = chart_data_en["Vote"].map(vote_order_mapping)

# Creating a chart in English
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_area()
    .encode(
        x="Year:O",
        y=alt.Y("% of all votes:Q", axis=alt.Axis(format=".0%")),
        color=alt.Color(
            "Vote:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=["Year:O", "Vote:N", alt.Tooltip("% of all votes:Q", format=".0%")],
    )
)
chart.save(".charts/figure_01_en.html")
chart

alt.Chart(...)

## Figure 2

INSERT CHART WITH TOTAL % OF VOTES FOR/AGAINST/ETC BY PARTY

In [10]:
# Aggregating data for chart
id_cols = ["PartiGruppe", "Stemme"]
sort_cols = ["PartiGruppe", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afsteminger[ft_afsteminger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("PartiGruppe")["Sæson"].transform(
    "count"
)
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)
chart_data = chart_data.sort_values(by=sort_cols)

In [11]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% af alle stemmer", "PartiGruppe": "Partigruppe"}
chart_data_dk = chart_data.rename(columns=col_names)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Imod", "Hverken for eller imod"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_dk["CustomOrder"] = chart_data_dk["Stemme"].map(vote_order_mapping)

# Creating a chart in Danish
chart = (
    alt.Chart(chart_data_dk, width="container")
    .mark_bar()
    .encode(
        x=alt.X("% af alle stemmer:Q", axis=alt.Axis(format=".0%")),
        y="Partigruppe",
        color=alt.Color(
            "Stemme:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=[
            "Partigruppe:O",
            "Stemme:N",
            alt.Tooltip("% af alle stemmer:Q", format=".0%"),
        ],
    )
)
chart.save(".charts/figure_02_dk.html")
chart

alt.Chart(...)

In [13]:
# Renaming columns for chart
col_names = {"AndelStemmer": "% of all votes", "PartiGruppe": "Party", "Stemme": "Vote"}
chart_data_en = chart_data.rename(columns=col_names)

# Replacing certain values
val_mapping = {
    "For": "For",
    "Imod": "Against",
    "Hverken for eller imod": "Neither for nor against",
}
chart_data_en["Vote"] = chart_data_en["Vote"].replace(val_mapping)

# Specifying custom sort order for the color variable
custom_val_order = ["For", "Against", "Neither for nor against"]
vote_order_mapping = {v: i for i, v in enumerate(custom_val_order)}
chart_data_en["CustomOrder"] = chart_data_en["Vote"].map(vote_order_mapping)

# Creating a chart in Danish
chart = (
    alt.Chart(chart_data_en, width="container")
    .mark_bar()
    .encode(
        x=alt.X("% of all votes:Q", axis=alt.Axis(format=".0%")),
        y="Party",
        color=alt.Color(
            "Vote:N",
            scale=alt.Scale(domain=custom_val_order, range=my_custom_palette),
        ),
        order=alt.Order("CustomOrder:Q"),
        tooltip=[
            "Party:O",
            "Vote:N",
            alt.Tooltip("% of all votes:Q", format=".0%"),
        ],
    )
)
chart.save(".charts/figure_02_en.html")
chart

alt.Chart(...)

In [14]:
print("DONE.")

DONE.
